# XGBoost model

In [47]:
import pandas as pd
import numpy as np


from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.metrics import confusion_matrix, classification_report
from xgboost import XGBClassifier
from sklearn.preprocessing import StandardScaler

In [135]:
!pwd

/home/jovana/code/y01a-BG/NeuroPredict


In [83]:
data = pd.read_csv('./raw_data/Epileptic Seizure Recognition.csv')

In [55]:
data.head()

,Unnamed,X1,X2,X3,X4,X5,X6,X7,X8,X9,...,X170,X171,X172,X173,X174,X175,X176,X177,X178,y
0,X21.V1.791,135,190,229,223,192,125,55,-9,-33,...,-17,-15,-31,-77,-103,-127,-116,-83,-51,4
1,X15.V1.924,386,382,356,331,320,315,307,272,244,...,164,150,146,152,157,156,154,143,129,1
2,X8.V1.1,-32,-39,-47,-37,-32,-36,-57,-73,-85,...,57,64,48,19,-12,-30,-35,-35,-36,5
3,X16.V1.60,-105,-101,-96,-92,-89,-95,-102,-100,-87,...,-82,-81,-80,-77,-85,-77,-72,-69,-65,5
4,X20.V1.54,-9,-65,-98,-102,-78,-48,-16,0,-21,...,4,2,-12,-32,-41,-65,-83,-89,-73,5


In [60]:
# # Andy

# data.drop(columns=['Unnamed'], inplace=True)
# data = data[data['y'].isin([1, 2, 3])]



# # Data preparation: extract features and adjust labels to start from 0
# X = data.drop(columns=['y']).values
# y = data['y'].values - 1  # Labels will now be 0, 1, and 2

# # Splitting dataset into training and testing sets (with stratification)
# X_train, X_test, y_train, y_test = train_test_split(
#     X, y, stratify=y, test_size=0.2, random_state=42
# )

# # Feature scaling for optimal performance
# scaler = StandardScaler()
# X_train = scaler.fit_transform(X_train)
# X_test = scaler.transform(X_test)

AttributeError: 'numpy.ndarray' object has no attribute 'head'

In [84]:
# preprocessor_01
data.drop(columns=['Unnamed'], inplace=True)
data = data[~data['y'].isin([4, 5])]

In [67]:
data.head()

,X1,X2,X3,X4,X5,X6,X7,X8,X9,X10,...,X170,X171,X172,X173,X174,X175,X176,X177,X178,y
1,386,382,356,331,320,315,307,272,244,232,...,164,150,146,152,157,156,154,143,129,1
7,1,-2,-8,-11,-12,-17,-15,-16,-18,-17,...,-79,-91,-97,-88,-76,-72,-66,-57,-39,2
8,-278,-246,-215,-191,-177,-167,-157,-139,-118,-92,...,-400,-379,-336,-281,-226,-174,-125,-79,-40,1
11,-167,-230,-280,-315,-338,-369,-405,-392,-298,-140,...,423,434,416,374,319,268,215,165,103,1
12,92,49,0,-32,-51,-65,-37,-19,-25,-29,...,-56,-41,-40,-43,-32,-13,-1,-7,-44,3


In [85]:
# test_train_and_split_02
X = data.drop(columns = 'y')
y = data.y

In [86]:
X = X.values
y = y.values
X.shape

(6900, 178)

In [87]:
####### test_and_train_split_02
import sklearn 
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,y)
X_train.shape, y_train.shape

((5175, 178), (5175,))

In [88]:
print(type(X_train))
print(X_train.dtypes if hasattr(X_train, "dtypes") else "Not a DataFrame")  
print(np.isnan(X_train).sum())  # Check for NaN values
type(X_train)

<class 'numpy.ndarray'>
Not a DataFrame
0


numpy.ndarray

In [89]:
X_test.shape

(1725, 178)

In [90]:
# encoding_03
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

## Initialize the model XGBoost

In [96]:
import tensorflow as tf
from keras import callbacks
from keras.callbacks import EarlyStopping

2025-03-21 13:20:02.980848: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-03-21 13:20:03.219268: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-03-21 13:20:03.234319: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2025-03-21 13:20:03.234368: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not ha

In [130]:
best_params = {
        'subsample': 0.8,
        'reg_lambda': 1,
        'reg_alpha': 1.0,
        'n_estimators': 300,
        'max_depth': 9,
        'learning_rate': 0.05,
        'gamma': 0,
        'colsample_bytree': 0.8
        }


xgb = XGBClassifier(
            subsample=best_params['subsample'],
            reg_lambda=best_params['reg_lambda'],
            reg_alpha=best_params['reg_alpha'],
            n_estimators=best_params['n_estimators'],
            max_depth=best_params['max_depth'],
            learning_rate=best_params['learning_rate'],
            gamma=best_params['gamma'],
            colsample_bytree=best_params['colsample_bytree'],

            objective='multi:softprob',
            eval_metric="mlogloss",
            num_class=3,
            verbosity=1,
            random_state=42,
            use_label_encoder=False
        )

In [131]:
xgb

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=0.8, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric='mlogloss',
              feature_types=None, feature_weights=None, gamma=0,
              grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=0.05, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=9, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=300, n_jobs=None, num_class=3, ...)

## Train the XGBoost

In [102]:
X_train, X_val, y_train, y_val = train_test_split(X_train_enc, y_train, test_size=0.2, random_state=42)

In [118]:
#y = y-1
#y_train= y_train - 1
#y_val = y_val - 1
#y_test = y_test-1

In [132]:
xgb.fit(
        X_train,
        y_train,
        eval_set=[(X_train, y_train), (X_val, y_val)],
        verbose=True
    )


/home/jovana/.pyenv/versions/3.10.6/envs/NeuroPredict/lib/python3.10/site-packages/xgboost/training.py:183: UserWarning: [15:11:02] WARNING: /workspace/src/learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)


[0]	validation_0-mlogloss:1.08048	validation_1-mlogloss:1.09977
[1]	validation_0-mlogloss:1.06091	validation_1-mlogloss:1.09936
[2]	validation_0-mlogloss:1.04506	validation_1-mlogloss:1.09965
[3]	validation_0-mlogloss:1.03049	validation_1-mlogloss:1.10136
[4]	validation_0-mlogloss:1.01211	validation_1-mlogloss:1.10196
[5]	validation_0-mlogloss:0.99676	validation_1-mlogloss:1.10167
[6]	validation_0-mlogloss:0.98015	validation_1-mlogloss:1.10109
[7]	validation_0-mlogloss:0.96511	validation_1-mlogloss:1.10140
[8]	validation_0-mlogloss:0.95328	validation_1-mlogloss:1.10243
[9]	validation_0-mlogloss:0.94087	validation_1-mlogloss:1.10276
[10]	validation_0-mlogloss:0.92553	validation_1-mlogloss:1.10353
[11]	validation_0-mlogloss:0.90930	validation_1-mlogloss:1.10438
[12]	validation_0-mlogloss:0.89574	validation_1-mlogloss:1.10367
[13]	validation_0-mlogloss:0.88074	validation_1-mlogloss:1.10339
[14]	validation_0-mlogloss:0.86616	validation_1-mlogloss:1.10432
[15]	validation_0-mlogloss:0.85431	

[126]	validation_0-mlogloss:0.22439	validation_1-mlogloss:1.15978
[127]	validation_0-mlogloss:0.22201	validation_1-mlogloss:1.15989
[128]	validation_0-mlogloss:0.22018	validation_1-mlogloss:1.16039
[129]	validation_0-mlogloss:0.21839	validation_1-mlogloss:1.16061
[130]	validation_0-mlogloss:0.21648	validation_1-mlogloss:1.16149
[131]	validation_0-mlogloss:0.21456	validation_1-mlogloss:1.16235
[132]	validation_0-mlogloss:0.21287	validation_1-mlogloss:1.16229
[133]	validation_0-mlogloss:0.21095	validation_1-mlogloss:1.16286
[134]	validation_0-mlogloss:0.20910	validation_1-mlogloss:1.16296
[135]	validation_0-mlogloss:0.20714	validation_1-mlogloss:1.16314
[136]	validation_0-mlogloss:0.20562	validation_1-mlogloss:1.16342
[137]	validation_0-mlogloss:0.20404	validation_1-mlogloss:1.16340
[138]	validation_0-mlogloss:0.20215	validation_1-mlogloss:1.16418
[139]	validation_0-mlogloss:0.20042	validation_1-mlogloss:1.16466
[140]	validation_0-mlogloss:0.19858	validation_1-mlogloss:1.16515
[141]	vali

[251]	validation_0-mlogloss:0.09898	validation_1-mlogloss:1.20806
[252]	validation_0-mlogloss:0.09856	validation_1-mlogloss:1.20819
[253]	validation_0-mlogloss:0.09812	validation_1-mlogloss:1.20884
[254]	validation_0-mlogloss:0.09768	validation_1-mlogloss:1.20909
[255]	validation_0-mlogloss:0.09727	validation_1-mlogloss:1.20931
[256]	validation_0-mlogloss:0.09683	validation_1-mlogloss:1.20941
[257]	validation_0-mlogloss:0.09640	validation_1-mlogloss:1.20942
[258]	validation_0-mlogloss:0.09598	validation_1-mlogloss:1.20941
[259]	validation_0-mlogloss:0.09556	validation_1-mlogloss:1.20967
[260]	validation_0-mlogloss:0.09512	validation_1-mlogloss:1.21018
[261]	validation_0-mlogloss:0.09471	validation_1-mlogloss:1.21076
[262]	validation_0-mlogloss:0.09429	validation_1-mlogloss:1.21084
[263]	validation_0-mlogloss:0.09387	validation_1-mlogloss:1.21113
[264]	validation_0-mlogloss:0.09353	validation_1-mlogloss:1.21155
[265]	validation_0-mlogloss:0.09312	validation_1-mlogloss:1.21221
[266]	vali

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=0.8, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric='mlogloss',
              feature_types=None, feature_weights=None, gamma=0,
              grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=0.05, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=9, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=300, n_jobs=None, num_class=3, ...)

## Evaluating the model

In [133]:
y_pred = xgb.predict(X_test)

In [134]:
y_pred

array([0, 0, 0, ..., 0, 2, 1])

In [125]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
from sklearn.metrics import classification_report

# Calculate accuracy score
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred, average='macro')  # 'macro' averages metrics across all classes
recall = recall_score(y_test, y_pred, average='macro')
f1 = f1_score(y_test, y_pred, average='macro')

# Display results as a table
results = {
        'Metric': ['Accuracy', 'Precision (Macro)', 'Recall (Macro)', 'F1-Score (Macro)'],
        'Score': [accuracy, precision, recall, f1]
    }
# Create DataFrame for easy display
results_df = pd.DataFrame(results)

# Confusion matrix
conf_matrix = confusion_matrix(y_test, y_pred)
print("✅ \nConfusion Matrix:")
print(conf_matrix)
# Create classification report (including precision, recall, f1-score per class)
class_report = classification_report(y_test, y_pred, target_names=['Class 0', 'Class 1', 'Class 2'])
print("✅ \nClassification Report:")
print(class_report)


print(f"✅ Metrics Table:{results_df}")

✅ 
Confusion Matrix:
[[174 197 208]
 [196 194 173]
 [178 215 190]]
✅ 
Classification Report:
              precision    recall  f1-score   support

     Class 0       0.32      0.30      0.31       579
     Class 1       0.32      0.34      0.33       563
     Class 2       0.33      0.33      0.33       583

    accuracy                           0.32      1725
   macro avg       0.32      0.32      0.32      1725
weighted avg       0.32      0.32      0.32      1725

✅ Metrics Table:              Metric     Score
0           Accuracy  0.323478
1  Precision (Macro)  0.323467
2     Recall (Macro)  0.323667
3   F1-Score (Macro)  0.323327


In [128]:
predictions = []
for label in y_pred:
    if label == 0:
        prediction_text = "Your EEG is predicted to be tumor-induced seizure EEG."
        predictions.append(prediction_text)
    elif label == 1:
        prediction_text = "Your EEG is predicted to be tumor baseline EEG."
        predictions.append(prediction_text)
    else:
        prediction_text = "Your EEG is predicted to be healthy baseline EEG."
        predictions.append(prediction_text)

print(predictions)
pred_dictionary = {"predictions": predictions}
print(f'✅ Prediction results for 6 EEG samples are : {pred_dictionary}')
return pred_dictionary
pred_dictionary

['Your EEG is predicted to be tumor-induced seizure EEG.', 'Your EEG is predicted to be tumor-induced seizure EEG.', 'Your EEG is predicted to be tumor-induced seizure EEG.', 'Your EEG is predicted to be healthy baseline EEG.', 'Your EEG is predicted to be tumor-induced seizure EEG.', 'Your EEG is predicted to be tumor-induced seizure EEG.', 'Your EEG is predicted to be tumor-induced seizure EEG.', 'Your EEG is predicted to be healthy baseline EEG.', 'Your EEG is predicted to be tumor baseline EEG.', 'Your EEG is predicted to be tumor baseline EEG.', 'Your EEG is predicted to be tumor baseline EEG.', 'Your EEG is predicted to be tumor baseline EEG.', 'Your EEG is predicted to be healthy baseline EEG.', 'Your EEG is predicted to be tumor-induced seizure EEG.', 'Your EEG is predicted to be tumor-induced seizure EEG.', 'Your EEG is predicted to be tumor baseline EEG.', 'Your EEG is predicted to be tumor baseline EEG.', 'Your EEG is predicted to be tumor-induced seizure EEG.', 'Your EEG is

SyntaxError: 'return' outside function (1612935276.py, line 16)

In [129]:
import xgboost
print(xgboost.__version__)

3.0.0
